In [507]:
import pandas as pd

In [546]:
pd.set_option('display.max_rows', 15)

In [509]:
df_all = pd.read_csv('../data/df_all.csv').drop(['Unnamed: 0', 'id', 'cluster', 'sentiment'], axis=1)

In [510]:
df_all

,question,answer,count
0,Что вы сможете использовать в своей работе?,"формулировать проблему, 5 поче",1.0
1,Что вы сможете использовать в своей работе?,поиск потерь,1.0
2,Что вы сможете использовать в своей работе?,правило 5 «почему» :),1.0
3,Что вы сможете использовать в своей работе?,умение работать в команде,1.0
4,Что вы сможете использовать в своей работе?,картирование и анализ,1.0
...,...,...,...
29659,•Где вы хотите провести следующую HR-сессию?,в москве,1.0
29660,•Где вы хотите провести следующую HR-сессию?,в сочи,1.0
29661,•Где вы хотите провести следующую HR-сессию?,сочи,1.0
29662,•Где вы хотите провести следующую HR-сессию?,окбм,1.0


In [511]:
index = pd.MultiIndex.from_arrays(df_all.iloc[:, :-1].values.T, names=('question', 'answer'))
df_grouped = pd.DataFrame({'count' : df_all.iloc[:, -1:].values.T[0]}, index=index)

In [512]:
df_grouped

count
question                                     answer                               
Что вы сможете использовать в своей работе?  формулировать проблему, 5 поче    1.0
                                             поиск потерь                      1.0
                                             правило 5 «почему» :)             1.0
                                             умение работать в команде         1.0
                                             картирование и анализ             1.0
...                                                                            ...
•Где вы хотите провести следующую HR-сессию? в москве                          1.0
                                             в сочи                            1.0
                                             сочи                              1.0
                                             окбм                              1.0
                                             сочи                              1.0

[29664 rows x 1 columns]

In [513]:
def get_abbreviations(f):
    abbreviations = dict()
    with open(f) as file:
        for line in file.readlines():
            key, value = map(str.strip, line.split('->'))
            abbreviations[key] = value
    return abbreviations

In [514]:
def replace_abbreviations(sentence):
    abbreviations = get_abbreviations('../data/слова.txt')
    words = sentence.lower().split()
    for i in range(len(words)):
        if words[i] in abbreviations:
            words[i] = abbreviations[words[i]]
    return ' '.join(words)

In [515]:
replace_abbreviations('я не оч, все раб. время в it отдыхаю, чил, зе бест')

'я не оч, все работа время в информационные технологии отдыхаю, чил, зе отлично'

In [516]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [517]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [518]:
def get_keys(list_of_dicts):
    keys = []
    for d in list_of_dicts:
        for key in d:
            keys.append(key)
    return keys

In [519]:
messages = df_all['answer'].values
list_of_sentiments = model.predict(messages, k=1)
df_all['sentiment'] = [sentiment if sentiment != 'speech' else 'positive' for sentiment in get_keys(list_of_sentiments)]

In [520]:
model.predict(['сила выносливость'], k=3)

[{'skip': 0.5621865391731262,
  'neutral': 0.2689514458179474,
  'negative': 0.051855459809303284}]

In [521]:
df_all[df_all['sentiment'] == 'skip']

,question,answer,count,sentiment
132,"Что для вас ""культура доверия"" в организации?",отсутствие наказаний за ошибки,1.0,skip
198,Что хорошего сделал за день?,макет,1.0,skip
360,Ваша главная эмоции сейчас?,страшно чутка,1.0,skip
449,"Считаете ли вы, что ТТТ могут помочь?","да, но…",1.0,skip
451,"Считаете ли вы, что ТТТ могут помочь?","да, но",1.0,skip
...,...,...,...,...
29515,Кто вы по профессии?,ведьма,1.0,skip
29518,Кто вы по профессии?,менеджер,1.0,skip
29529,Кто я?,робота,1.0,skip
29535,Кто я?,харошо,1.0,skip


In [532]:
df_all['corrected'] = list(map(lambda answer: replace_abbreviations(answer), df_all['answer'].values))

In [533]:
df_all

,question,answer,count,sentiment,corrected
0,Что вы сможете использовать в своей работе?,"формулировать проблему, 5 поче",1.0,neutral,"формулировать проблему, 5 поче"
1,Что вы сможете использовать в своей работе?,поиск потерь,1.0,neutral,поиск потерь
2,Что вы сможете использовать в своей работе?,правило 5 «почему» :),1.0,neutral,правило 5 «почему» :)
3,Что вы сможете использовать в своей работе?,умение работать в команде,1.0,neutral,умение работать в команде
4,Что вы сможете использовать в своей работе?,картирование и анализ,1.0,neutral,картирование и анализ
...,...,...,...,...,...
29659,•Где вы хотите провести следующую HR-сессию?,в москве,1.0,neutral,в москве
29660,•Где вы хотите провести следующую HR-сессию?,в сочи,1.0,neutral,в сочи
29661,•Где вы хотите провести следующую HR-сессию?,сочи,1.0,neutral,сочи
29662,•Где вы хотите провести следующую HR-сессию?,окбм,1.0,neutral,окбм


In [534]:
messages = df_all['corrected'].values
list_of_sentiments = model.predict(messages, k=1)
df_all['sentiment_corr'] = [sentiment if sentiment != 'speech' else 'positive' for sentiment in get_keys(list_of_sentiments)]

In [545]:
df_all[df_all['sentiment'] != df_all['sentiment_corr']][['answer','sentiment','corrected','sentiment_corr']]

,answer,sentiment,corrected,sentiment_corr
360,страшно чутка,skip,страшно немного,negative
520,стабильная и достойная з/п,negative,стабильная и достойная зарплата,neutral
614,тип лидеров по адизесу,negative,тип лидеров программное обеспечение адизесу,neutral
923,подходы к лидерству,neutral,подходы к лидерство,negative
1458,#росатомсила,skip,крутая госкорпорация сила,positive
1593,гуд,neutral,хорошо,positive
1622,бамбалейла,skip,отлично,positive
1945,росатом,neutral,крутая госкорпорация,positive
1961,росатом прогрессирует,neutral,крутая госкорпорация прогрессирует,positive
1973,росатом - люди,skip,крутая госкорпорация - люди,positive


In [523]:
from spellchecker import SpellChecker

spell = SpellChecker(language='ru')

In [468]:
print(spell.candidates('оформл'))

{'формы', 'форма', 'форму', 'форме'}
